In [ ]:
exp = !!!exp!!!
branch = !!!branch!!!
git_user = !!!git_user!!!
git_repo = !!!git_repo!!!
wandb_flag = !!!wandb_flag!!!
output_dir = !!!output_dir!!!
dataset_files = !!!dataset_files!!!

# Clone git repo

In [ ]:
%cd ~
!git clone https://github.com/$git_user/$git_repo >/dev/null
%cd $git_repo
!git checkout $branch
# !pip install -e .
!pip install torch_geometric

# Setup weights and biases

In [ ]:
if wandb_flag:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    wandb_api_key = user_secrets.get_secret("wandb_api_key")

    !pip install wandb >/dev/null
    !wandb login $wandb_api_key

# Unzip dataset

In [ ]:
#Uniquement sur Kaggle
import shutil
import os
import multiprocessing
import threading

def copy_file(src_file, dest_file):
    shutil.copy(src_file, dest_file)

def copy_files_with_threads(src_dir, dest_dir, num_threads=4):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    file_list = os.listdir(src_dir)
    num_files = len(file_list)
    chunk_size = num_files // num_threads
    threads = []
    for i in range(num_threads):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_threads - 1 else num_files
        chunk_files = file_list[start:end]

        thread = threading.Thread(target=copy_files_in_chunk, args=(src_dir, dest_dir, chunk_files))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()

def copy_files_in_chunk(src_dir, dest_dir, file_list):
    for filename in file_list:
        src_file = os.path.join(src_dir, filename)
        dest_file = os.path.join(dest_dir, filename)
        copy_file(src_file, dest_file)

for filename in os.listdir("/kaggle/input/datanlp/data"):
    if filename not in "raw":
        src_file = os.path.join("/kaggle/input/datanlp/data", filename)
        dest_file = os.path.join("/kaggle/working", filename)
        shutil.copy(src_file, dest_file)

num_threads = multiprocessing.cpu_count()
print("Number of CPU threads available:", num_threads)

src_dir = "/kaggle/input/datanlp/data/raw"
dest_dir = "/kaggle/working/raw"

copy_files_with_threads(src_dir, dest_dir, num_threads-1)

In [ ]:
if dataset_files is not None:
    for dataset_file in dataset_files:
        !tar -xzf /kaggle/input/$dataset_file

# Launch training

In [ ]:
exp_str = ' '.join([str(e) for e in exp])
wb_ext = "-nowb" if not wandb_flag else ""
!python train.py -e $exp_str $wb_ext

# Prepare outputs

In [ ]:
if output_dir is not None:
    !unzip /kaggle/working/output.zip -d $output_dir